# Example of import_raster cmd using API of geonode_conabio and [kale](https://github.com/kubeflow-kale/kale) functionality

References:

https://github.com/CONABIO/geonode/blob/master/notebooks/towards_module_python_to_ingest_programmatically_into_geonode/1_normalizing_and_reprojecting.ipynb

https://github.com/CONABIO/kube_sipecam_playground/blob/master/MAD_Mex/notebooks/1_issues_and_nbooks/1_issue_8_pipeline_upload_lc_result_to_geonode_using_kale.ipynb

https://github.com/CONABIO/antares3/blob/develop/madmex/management/commands/db_to_raster.py



**Raster has small size**

In [1]:
import os
import glob
import json

from pyproj import Proj
from rasterio.merge import merge
import rasterio
from rasterio.warp import transform_geom
from rasterio import features
from docker import APIClient

from geonode_conabio.wrappers import reproj_and_write_one_band_raster
from geonode_conabio.utils_docker import import_layers_via_docker

from madmex.models import Country, Region

In [2]:
#next to import: from madmex.models import Country, Region
#!pip3 install django
#!pip3 install djangorestframework --quiet
#!pip3 install django-cors-headers --quiet
#!pip3 install psycopg2-binary --quiet
#!git clone --single-branch -b develop https://github.com/CONABIO/antares3.git /root/antares3
#!cp /shared_volume/.antares /root/antares3

In [6]:
#next to import: from madmex.models import Country, Region
os.chdir("/root/antares3/")
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "madmex.settings")
import django
django.setup()

In [20]:
path = "/shared_volume/land_cover_results_parallel"

filenames = [f for f in glob.glob(os.path.join(path, "*.tif")) if not "geonode" in f]

In [21]:
filenames

['/shared_volume/land_cover_results_parallel/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_49_-41.tif',
 '/shared_volume/land_cover_results_parallel/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_49_-39.tif',
 '/shared_volume/land_cover_results_parallel/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_49_-40.tif']

In [22]:
output_filenames = [f.replace(".tif", "_processed_for_geonode.tif") for f in filenames]

In [23]:
output_filenames

['/shared_volume/land_cover_results_parallel/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_49_-41_processed_for_geonode.tif',
 '/shared_volume/land_cover_results_parallel/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_49_-39_processed_for_geonode.tif',
 '/shared_volume/land_cover_results_parallel/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_49_-40_processed_for_geonode.tif']

# Reproject and write to FS

In [24]:
for idx, filename in enumerate(filenames):
    with rasterio.open(filename) as src:
        src_crs = src.crs.to_string()
        proj_crs = Proj(src_crs)
        if not proj_crs.crs.is_geographic:
            reproj_and_write_one_band_raster(src, output_filenames[idx],
                                             is_geographic=False)
        else:
            reproj_and_write_one_band_raster(src, output_filenames[idx])

# Merge

In [25]:
src_files_to_mosaic = [rasterio.open(f) for f in output_filenames]
# Retrieve metadata of one file for later use
meta = src_files_to_mosaic[0].meta.copy()
mosaic, out_trans = merge(src_files_to_mosaic)

In [26]:
meta.update(width=mosaic.shape[2],
            height=mosaic.shape[1],
            transform=out_trans,
            compress="lzw")

In [28]:
os.environ.setdefault("DJANGO_ALLOW_ASYNC_UNSAFE", "true")
region = "Chiapas"
# Query country or region contour
try:
    region = Country.objects.get(name=region).the_geom
except Country.DoesNotExist:
    region = Region.objects.get(name=region).the_geom

In [29]:
region

In [31]:
region_geojson = region.geojson
geometry_region = json.loads(region_geojson)

In [35]:
# rasterize region using mosaic as template
mask_array = features.rasterize(shapes=[(geometry_region, 1)],
                                out_shape=(mosaic.shape[1],mosaic.shape[2]),
                                fill=0,
                                transform=meta['transform'],
                                dtype=rasterio.uint8)

In [36]:
# Apply mask to mosaic
mosaic[:,mask_array==0] = 0

In [37]:
# Write results to file
filename = "raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_merge.tif"
filename_mosaic = os.path.join(path, filename)

In [38]:
filename_mosaic

'/shared_volume/land_cover_results_parallel/raster_landsat8_chiapas_madmex_31_clases_pixel_wise_via_kale_merge.tif'

In [39]:
with rasterio.open(filename_mosaic, "w", **meta) as dst:
    dst.write(mosaic)

In [ ]:
region = "".join(["\"Chiapas, Mexico, North America, Latin America\""])
name = "\"Chiapas_lc_2017_landsat8_test merge\""
title = "\"Land cover Chiapas landsat8 2017 merge test using kale\""
abstract = "\"Test merge kale\""
kw = "\"Chiapas\""


c = APIClient(base_url='tcp://172.17.0.1:1111')
cmd = "".join(["python manage.py importlayers -v 3 -i -o ",
               "-n ", name, " ",
               "-t ", title, " ",
               "-a ", abstract, " ",
               "-k ", kw, " ",
               "-r ", region, " ",
               filename_mosaic, " ",
               "--settings=geonode.local_settings"])
ex = c.exec_create(container = 'spcgeonode_django_1', 
                   cmd = cmd)
ex_start = c.exec_start(exec_id=ex)

print(ex_start)
c.close()
#close & clean:
#os.remove(output_filename)
#for idx, of in output_filenames:
#    src_files_to_mosaic[idx].close()
#    os.remove(of)